# Language Classification Problem

For this problem first lets check data.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

In [2]:
english_data = pd.read_csv("english.csv")
turkish_data = pd.read_csv("turkish.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'english.csv'

In [ ]:
english_data.head()

In [ ]:
turkish_data.head()

When I looked the head of data I detect some of abbreviations for english dataset.After checking full of data I detect many more abbreviations. And as for turkish data We can't see verb suffix as "yor" "dı" "tı" but we can see "mak" for verbs. After these observations I think best way to approuch this kind of data is character N-gram. But before we generating model lets split data for testing.

In [3]:
from sklearn.model_selection import train_test_split

turkish_train, = train_test_split(df, test_size=0.2, random_state=25)

NameError: name 'df' is not defined

# Lineer Regresion Model Building

//

//

//

## Comments

////

/////

# Comments About Scores

///

### Regressions Analysis

///

///

## ///

### Comments
//

# Summary

///